# Get combined mechanisms data

In [1]:
import numpy, os
import pandas as pd

## Read the data

In [2]:
MODEL_NAME = "llama2_7B" # "gpt2-xl"

In [3]:
datasets = {"gpt2-xl": [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_exact.json",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_2377617/cases",
             "filename_template": "knowledge_{}_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_guesswork.json",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_guesswork_2388522/cases",
              "filename_template": "knowledge_{}_mlp.npz"
              }
             ],
            "llama2_7B": [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_7B/1000_exact.jsonl",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_7B/causal_trace_exact_2398046/cases",
             "filename_template": "{}_candidate_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_7B/1000_guesswork.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_7B/causal_trace_guesswork_2398048/cases",
              "filename_template": "{}_candidate_mlp.npz"
              }
             ],
            }

data_shares = {"exact_recall": 0.33,
               "prompt_bias": 0.17,
               "person_name_bias": 0.17,
               "guesswork": 0.33
              }
assert sum(data_shares.values())==1

num_samples = 1000

In [4]:
data = pd.DataFrame()
for dataset in datasets[MODEL_NAME]:
    tmp_data = pd.read_json(dataset["queries_file"], lines=dataset["queries_file"].endswith("jsonl")).sample(
        int(num_samples*data_shares[dataset["name"]]), random_state=42)
    tmp_data["type"] = dataset["name"]
    tmp_data["CT_results_dir"] = dataset["CT_results_dir"]
    tmp_data["filename_template"] = dataset["filename_template"]
    data = pd.concat((data, tmp_data), ignore_index=True)
    
data

,obj_label,sub_label,predicate_id,source,prompt,template,sub_view_rates,obj_view_rates,answers,p_answers,...,answers_for_PB_She,confidence,candidate_p,top10_tokens,prediction_p,prediction,confident_flag,answers_for_PB_It,answers_for_PB_The_city,answers_for_PB_The_organisation
0,Japan,Tsubasa: Reservoir Chronicle,P495,TREx_UHN,"Tsubasa: Reservoir Chronicle, formulated in","[X], formulated in [Y]",11929.083333,491464.833333,Japan,0.132291,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Milan,Isotta Fraschini,P740,TREx_UHN,Isotta Fraschini was started in,[X] was started in [Y],3228.166667,99240.000000,Milan,0.077576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Glasgow,Deacon Blue,P740,TREx_UHN,Deacon Blue was founded in,[X] was founded in [Y],11881.250000,91203.750000,Glasgow,0.407596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Philadelphia,Man Man,P740,TREx_UHN,Man Man was founded in,[X] was founded in [Y],3228.833333,170396.666667,Philadelphia,0.196482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,Floricienta,P495,TREx_UHN,Floricienta was created in,[X] was created in [Y],2886.833333,252651.083333,Argentina,0.041186,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Denmark,Lars Paaske,P27,TREx_UHN,"Lars Paaske, a citizen of","[X], a citizen of [Y]",214.000000,259110.416667,Norway,0.043228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Hungary,EZ Basic,P495,TREx_UHN,"EZ Basic, that originated in","[X], that originated in [Y]",134.666667,179669.250000,Japan,0.035749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Alexandria,Michel Pablo,P19,TREx_UHN,Michel Pablo was born in,[X] was born in [Y],664.333333,70454.083333,Paris,0.087699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Ireland,Pat Scully,P27,TREx_UHN,Pat Scully has a citizenship of,[X] has a citizenship of [Y],353.750000,269795.166667,Australia,0.09156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.type.value_counts()

guesswork           330
exact_recall        330
prompt_bias         170
person_name_bias    170
Name: type, dtype: int64

Save the data

In [6]:
data.to_json(f"/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/{MODEL_NAME}/1000_combined_mechanisms.json", orient="records")